<a href="https://colab.research.google.com/github/AlexFly666/002-openai-quickstart-jike-peng/blob/main/langchain/jupyter/autogpt/autogpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## AutoGPT 智能体

以下`SERPAPI_API_KEY`仅为示例，请访问 https://serpapi.com 注册账号并替换为自己的 `API_KEY`（每月100次免费调用）

In [22]:
!pip install -U langchain \
langchain-openai \
langchain-core \
langchain-community \
google-search-results \
faiss-cpu \
langchain_experimental

# Langchian v0.3版本
# !pip install \
#   langchain==0.3.18 \
#   langchain-openai==0.3.4 \
#   langchain-core==0.3.34 \
#   langchain-community==0.3.17 \
#   google-search-results==2.4.2 \
#   faiss-cpu==1.10.0 \
#   langchain_experimental==0.3.4

In [21]:
import os

os.environ["SERPAPI_API_KEY"] = "XXXX"

In [3]:
from langchain.utilities import SerpAPIWrapper
from langchain.agents import Tool
from langchain.tools.file_management.write import WriteFileTool
from langchain.tools.file_management.read import ReadFileTool


In [4]:
# 构造 AutoGPT 的工具集
search = SerpAPIWrapper()
tools = [
    Tool(
        name="search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions",
    ),
    WriteFileTool(),
    ReadFileTool(),
]

In [5]:
from langchain_openai import OpenAIEmbeddings

In [6]:
# OpenAI Embedding 模型
# embeddings_model = OpenAIEmbeddings()
# 词嵌入（代理方式）
# OpenAI 词嵌入（代理方式）
embeddings_model = OpenAIEmbeddings(
    api_key="XXXX",
    base_url="https://vip.apiyi.com/v1"
)


In [10]:
import faiss
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore

# OpenAI Embedding 向量维数
embedding_size = 1536
# 使用 Faiss 的 IndexFlatL2 索引
index = faiss.IndexFlatL2(embedding_size)
# 实例化 Faiss 向量数据库
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

## 实例化自主智能体 Auto-GPT

In [12]:
from langchain_experimental.autonomous_agents import AutoGPT
from langchain_openai import ChatOpenAI

In [17]:
#  OpenAI API调用（代理方式）
llm=ChatOpenAI(
    api_key="XXXX",
    base_url="https://vip.apiyi.com/v1",
    model_name="gpt-4o",
    temperature=0,
    verbose=True
)


agent = AutoGPT.from_llm_and_tools(
    ai_name="Jarvis",
    ai_role="Assistant",
    tools=tools,
    llm=llm,
    memory=vectorstore.as_retriever(
        search_type="similarity_score_threshold",
        search_kwargs={"score_threshold": 0.8}),# 实例化 Faiss 的 VectorStoreRetriever
)

In [18]:
# 打印 Auto-GPT 内部的 chain 日志
agent.chain.verbose = True

In [19]:
agent.run(["2023年成都大运会，中国金牌数是多少"])



> Entering new LLMChain chain...


Prompt after formatting:
System: You are Jarvis, Assistant
Your decisions must always be made independently without seeking user assistance.
Play to your strengths as an LLM and pursue simple strategies with no legal complications.
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. 2023年成都大运会，中国金牌数是多少


Constraints:
1. ~4000 word limit for short term memory. Your short term memory is short, so immediately save important information to files.
2. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.
3. No user assistance
4. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. search: useful for when you need to answer questions about current events. You should ask targeted questions, args json schema: {"tool_input": {"type": "string"}}
2. write_file: Write file to disk, args json schema: {"file_path": {"description": "name of file", "ti

'China won 103 gold medals at the 2023 Chengdu Universiade.'